In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
query = ''' 
SELECT COUNT(*) AS diligent_students_count
FROM (
    SELECT 
        st_id, 
        COUNT(DISTINCT DATE_TRUNC('month', timest)) AS months_solved
    FROM peas
    WHERE correct = true
    GROUP BY st_id
    HAVING COUNT(*) >= 20
) AS diligent_students_subquery
'''

diligent_students = ph.read_clickhouse(query=query, connection=connection_default)
diligent_students

,diligent_students_count
0,136


In [4]:
query_ = ''' 
WITH information AS
    (
    SELECT 
        std.test_grp,
        SUM(fpc.money) AS sum_money,
        COUNT(DISTINCT std.st_id) AS users
    FROM final_project_check AS fpc
    RIGHT JOIN studs AS std ON fpc.st_id = std.st_id
    GROUP BY std.test_grp
    ),
information_2 AS
    (
    SELECT 
        test_grp, 
        COUNT (DISTINCT std.st_id) AS active_users
    FROM 
        (   
        SELECT
            std.test_grp,
            std.st_id,
            SUM(correct) AS tasks_completed
        FROM peas AS p
        RIGHT JOIN studs AS std ON p.st_id = std.st_id
        GROUP BY std.st_id, std.test_grp  
        HAVING tasks_completed > 10
        )
    GROUP BY test_grp
    ),
active_users_math AS
    (
    SELECT 
        test_grp, 
        COUNT (DISTINCT std.st_id) AS active_users_math
    FROM 
        (   
        SELECT
            std.test_grp,
            std.st_id,
            SUM(correct) AS tasks_completed
        FROM peas AS p
        RIGHT JOIN studs AS std ON p.st_id = std.st_id
        WHERE subject = 'Math'
        GROUP BY std.st_id, std.test_grp  
        HAVING tasks_completed > 2
        )
    GROUP BY test_grp
    ),
users_with_purchase AS 
    (
    SELECT
        std.test_grp AS test_grp,
        COUNT(DISTINCT fpc.st_id) AS users_purchase
    FROM final_project_check AS fpc
    LEFT JOIN studs AS std ON fpc.st_id = std.st_id
    GROUP BY std.test_grp
    ),
active_users_with_purchase AS 
    (
    SELECT
    std.test_grp AS test_grp,
    COUNT(DISTINCT fpc.st_id) AS active_users_purchase
FROM final_project_check AS fpc
LEFT JOIN studs AS std ON fpc.st_id = std.st_id
WHERE fpc.st_id IN (SELECT st_id FROM (
        SELECT
            std.st_id AS st_id,
            SUM(correct) AS tasks_completed
        FROM peas AS p
        RIGHT JOIN studs AS std ON p.st_id = std.st_id
        GROUP BY std.st_id  
        HAVING tasks_completed > 10
)
)
GROUP BY test_grp
    ),
active_users_with_purchase_math AS 
    (
    SELECT
        std.test_grp AS test_grp,
        COUNT(DISTINCT fpc.st_id) AS active_users_purchase_math
    FROM final_project_check AS fpc
    JOIN studs AS std ON fpc.st_id = std.st_id
    WHERE fpc.subject = 'Math' AND  
            fpc.st_id IN (SELECT st_id FROM 
            (
            SELECT
            std.st_id AS st_id,
            SUM(correct) AS tasks_completed
        FROM peas AS p
        RIGHT JOIN studs AS std ON p.st_id = std.st_id
        WHERE subject = 'Math'
        GROUP BY std.st_id 
        HAVING tasks_completed > 2
            ))
    GROUP BY std.test_grp
    ),
money_active AS 
    (
    SELECT
        a.test_grp AS test_grp,
        SUM(b.money) AS money_active_users
    FROM studs AS a
    LEFT JOIN (
        SELECT
            st_id,
            SUM(money) AS money
        FROM final_project_check 
        GROUP BY st_id) AS b 
    ON a.st_id = b.st_id
    LEFT JOIN (
        SELECT
            st_id,
            SUM(tasks_completed) AS score
        FROM (
            SELECT
                st_id,
                SUM(correct) AS tasks_completed
            FROM peas
            GROUP BY
                st_id
            ) 
        GROUP BY st_id) AS c
    ON a.st_id = c.st_id
GROUP BY test_grp
HAVING score > 10
    )
SELECT 
    a.test_grp AS group,
    ROUND(sum_money / users) AS ARPU, 
    ROUND(money_active_users / active_users) AS ARPAU,
    ROUND((users_purchase / users), 2) AS cr_to_purchase,
    ROUND((active_users_purchase / active_users), 2) AS cr_active_to_purchase,
    ROUND((active_users_purchase_math / active_users_math), 2) AS cr_math_to_purchase
FROM information AS a JOIN information_2 AS b ON a.test_grp = b.test_grp JOIN users_with_purchase AS c ON a.test_grp = c.test_grp JOIN money_active AS d ON a.test_grp = d.test_grp JOIN active_users_math AS f ON a.test_grp = f.test_grp JOIN active_users_with_purchase AS g ON a.test_grp = g.test_grp JOIN active_users_with_purchase_math AS h ON a.test_grp = h.test_grp 
'''

funnel_optimization = ph.read_clickhouse(query=query_, connection=connection_default)
funnel_optimization

,group,ARPU,ARPAU,cr_to_purchase,cr_active_to_purchase,cr_math_to_purchase
0,control,4541.0,10394.0,0.05,0.11,0.06
1,pilot,11508.0,29740.0,0.11,0.26,0.10
